In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [3]:
sc = SparkContext()

In [4]:
spark = SparkSession.builder.getOrCreate()

In [5]:
ss = spark

In [6]:
l = [('Alice',1)]
ss.createDataFrame(l,['name','age']).collect()

[Row(name='Alice', age=1)]

In [7]:
rdd = sc.parallelize(l)
spark.createDataFrame(rdd).collect()
df = spark.createDataFrame(rdd,['name','age'])
df.show()

+-----+---+
| name|age|
+-----+---+
|Alice|  1|
+-----+---+



In [8]:
from pyspark.sql import Row
Person = Row('name','age')
person = rdd.map(lambda r: Person(*r))
df2 = spark.createDataFrame(person)
df2.show()

+-----+---+
| name|age|
+-----+---+
|Alice|  1|
+-----+---+



In [9]:
from pyspark.sql.types import *
schema = StructType([
    StructField("name",StringType(),True),
    StructField("age",IntegerType(),True)
])
df3 = spark.createDataFrame(rdd, schema)
df3.show()

+-----+---+
| name|age|
+-----+---+
|Alice|  1|
+-----+---+



In [10]:
ss.createDataFrame(df.toPandas()).show()

+-----+---+
| name|age|
+-----+---+
|Alice|  1|
+-----+---+



In [11]:
dfr = ss.read

In [12]:
df = dfr.csv('DataSets/StudentsPerformance.csv',header = True, inferSchema=True)

In [13]:
df.cache()
print(type(df))

<class 'pyspark.sql.dataframe.DataFrame'>


In [14]:
df.count()

1000

In [15]:
df.columns

['gender',
 'race/ethnicity',
 'parental level of education',
 'lunch',
 'test preparation course',
 'math score',
 'reading score',
 'writing score']

In [16]:
df.dtypes

[('gender', 'string'),
 ('race/ethnicity', 'string'),
 ('parental level of education', 'string'),
 ('lunch', 'string'),
 ('test preparation course', 'string'),
 ('math score', 'int'),
 ('reading score', 'int'),
 ('writing score', 'int')]

In [30]:
df.toPandas()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


In [17]:
df.select('gender','lunch').show()

+------+------------+
|gender|       lunch|
+------+------------+
|female|    standard|
|female|    standard|
|female|    standard|
|  male|free/reduced|
|  male|    standard|
|female|    standard|
|female|    standard|
|  male|free/reduced|
|  male|free/reduced|
|female|free/reduced|
|  male|    standard|
|  male|    standard|
|female|    standard|
|  male|    standard|
|female|    standard|
|female|    standard|
|  male|    standard|
|female|free/reduced|
|  male|free/reduced|
|female|free/reduced|
+------+------------+
only showing top 20 rows



In [18]:
df.agg({'math score':'max','reading score':'max','writing score':'max'}).show()
print()
df.agg({'math score':'min','reading score':'min','writing score':'min'}).show()
print()
df.agg({'math score':'mean','reading score':'mean','writing score':'mean'}).show()



+------------------+------------------+---------------+
|max(reading score)|max(writing score)|max(math score)|
+------------------+------------------+---------------+
|               100|               100|            100|
+------------------+------------------+---------------+


+------------------+------------------+---------------+
|min(reading score)|min(writing score)|min(math score)|
+------------------+------------------+---------------+
|                17|                10|              0|
+------------------+------------------+---------------+


+------------------+------------------+---------------+
|avg(reading score)|avg(writing score)|avg(math score)|
+------------------+------------------+---------------+
|            69.169|            68.054|         66.089|
+------------------+------------------+---------------+



In [19]:
df.where("gender=='female' and lunch like '%stan%'").show()

+------+--------------+---------------------------+--------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|   lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+--------+-----------------------+----------+-------------+-------------+
|female|       group B|          bachelor's degree|standard|                   none|        72|           72|           74|
|female|       group C|               some college|standard|              completed|        69|           90|           88|
|female|       group B|            master's degree|standard|                   none|        90|           95|           93|
|female|       group B|         associate's degree|standard|                   none|        71|           83|           78|
|female|       group B|               some college|standard|              completed|        88|           95|           92|
|female|

In [20]:
df.corr('math score','reading score')

0.8175796636720539

In [21]:
df.select('math score','reading score','writing score').describe().show()

+-------+------------------+------------------+-----------------+
|summary|        math score|     reading score|    writing score|
+-------+------------------+------------------+-----------------+
|  count|              1000|              1000|             1000|
|   mean|            66.089|            69.169|           68.054|
| stddev|15.163080096009454|14.600191937252223|15.19565701086966|
|    min|                 0|                17|               10|
|    max|               100|               100|              100|
+-------+------------------+------------------+-----------------+



In [22]:
df.groupBy('gender').mean().show()

+------+------------------+------------------+------------------+
|gender|   avg(math score)|avg(reading score)|avg(writing score)|
+------+------------------+------------------+------------------+
|female|63.633204633204635| 72.60810810810811| 72.46718146718146|
|  male| 68.72821576763485| 65.47302904564316| 63.31120331950208|
+------+------------------+------------------+------------------+



In [23]:
df.rdd.take(5)

[Row(gender='female', race/ethnicity='group B', parental level of education="bachelor's degree", lunch='standard', test preparation course='none', math score=72, reading score=72, writing score=74),
 Row(gender='female', race/ethnicity='group C', parental level of education='some college', lunch='standard', test preparation course='completed', math score=69, reading score=90, writing score=88),
 Row(gender='female', race/ethnicity='group B', parental level of education="master's degree", lunch='standard', test preparation course='none', math score=90, reading score=95, writing score=93),
 Row(gender='male', race/ethnicity='group A', parental level of education="associate's degree", lunch='free/reduced', test preparation course='none', math score=47, reading score=57, writing score=44),
 Row(gender='male', race/ethnicity='group C', parental level of education='some college', lunch='standard', test preparation course='none', math score=76, reading score=78, writing score=75)]

In [24]:
df.sample(fraction=0.02,seed=42).count()

17

In [25]:
df.select('math score','writing score','reading score').summary().show()

+-------+------------------+-----------------+------------------+
|summary|        math score|    writing score|     reading score|
+-------+------------------+-----------------+------------------+
|  count|              1000|             1000|              1000|
|   mean|            66.089|           68.054|            69.169|
| stddev|15.163080096009454|15.19565701086966|14.600191937252223|
|    min|                 0|               10|                17|
|    25%|                57|               57|                59|
|    50%|                66|               69|                70|
|    75%|                77|               79|                79|
|    max|               100|              100|               100|
+-------+------------------+-----------------+------------------+



In [26]:
df.createTempView('scores')

In [27]:
ss.sql("SELECT * from scores where gender = 'male'").show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|  male|       group A|         associate's degree|free/reduced|                   none|        47|           57|           44|
|  male|       group C|               some college|    standard|                   none|        76|           78|           75|
|  male|       group B|               some college|free/reduced|                   none|        40|           43|           39|
|  male|       group D|                high school|free/reduced|              completed|        64|           64|           67|
|  male|       group C|         associate's degree|    standard|                   none|        58|     

In [28]:
a = ss.sql("SELECT * from scores where gender = 'male'")

In [29]:
a.show()

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|  male|       group A|         associate's degree|free/reduced|                   none|        47|           57|           44|
|  male|       group C|               some college|    standard|                   none|        76|           78|           75|
|  male|       group B|               some college|free/reduced|                   none|        40|           43|           39|
|  male|       group D|                high school|free/reduced|              completed|        64|           64|           67|
|  male|       group C|         associate's degree|    standard|                   none|        58|     